
# Code for calculating the THERMAL LOAD required for heating swimming pools and sizing the number of flat solar collectors.

# Part I - Estimation of solar irradiation using the Cresesb or NASA website (Parte I - Estimativa da irradiação solar utilizando o site do Cresesb ou da Nasa)
# It is possible to determine the horizontal solar radiation using one of the following databases:

1) http://www.cresesb.cepel.br/index.php?section=sundata, add Geographic Coordinate and use the value for Horizontal Plane solar radiation.
2) https://power.larc.nasa.gov/data-access-viewer/, add Geographic Coordinate and use the value for Horizontal Plane solar radiation.

Reference: Chapter 9.7 SWIMMING POOL HEATING LOADS, Duffie and Beckman (2020)

In [ ]:
!pip install CoolProp
!pip install pvlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 37.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 31.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pvlib

# Criando uma série temporal com dados meteorológicos
tz = 'Etc/GMT+3'  # Fuso horário
#No Google maps, a latitude e a longitude podem ser obtidas clicando com o botão direito no local desejado. E utilizando o Google Earth, as informações de latitude e a longitude estão disponíveis no canto inferior direito da tela.
lat = -30.035409581927556  # Latitude. 
lon = -51.197064246714239 # Longitude.
alt = 33  # Altitude em metros. Utilizando o Google Earth, no canto inferior direito pode-se obter a altitude local.

# Definindo as características da superfície inclinada
surface_tilt = -30  # Inclinação em graus
surface_azimuth = 180  # Azimute em graus

date_range = pd.date_range(start='2022-01-01 00:00:00', end='2022-12-31 23:59:59', freq='1min', tz=tz)
solpos = pvlib.solarposition.get_solarposition(date_range, lat, lon)
dni_extra = pvlib.irradiance.get_extra_radiation(date_range)
pressure = pvlib.atmosphere.alt2pres(alt)
airmass = pvlib.atmosphere.get_relative_airmass(solpos['apparent_zenith'])
pressure_corr = pvlib.atmosphere.get_absolute_airmass(airmass, pressure)
linke_turbidity = pvlib.clearsky.lookup_linke_turbidity(date_range, lat, lon)
solar_zenith = solpos['apparent_zenith']
solar_azimuth = solpos['azimuth']
diffuse_frac = pvlib.irradiance.get_ground_diffuse(surface_tilt, solar_azimuth)

# Calculando a irradiação solar em superfície inclinada com o modelo de Perez
dni = pvlib.irradiance.get_total_irradiance(surface_tilt, surface_azimuth, solar_zenith, solar_azimuth,
                                            dni_extra, linke_turbidity, diffuse_frac, pressure_corr,
                                            model='perez')

# Obtendo a irradiância total em superfície inclinada a partir do dicionário retornado
irrad_total = dni['poa_global']

# Calculando a média da irradiância total na superfície
irrad_total_media = dni['poa_global'].mean()

# Imprimindo o valor da média da irradiância total na superfície em W/m²
print(f"Irradiância total média na superfície: {irrad_total_media:.2f} W/m²")

Irradiância total média na superfície: 423.29 W/m²


In [ ]:
# To use the value of horizontal plane (Irradiação solar diária média mensal [kWh/m2.dia])
HSP = 4.43 # kWh/m2.dia Data obtained from one of the sites above (CRESESB or NASA). Here, it was used the CRESESB DATA for Porto Alegr - RS. 
HSP_MJ = HSP*3.6 #Convert to MJ/m2.dia

print(f"Irradiância total média no plano horizontal é: {HSP_MJ:.2f} MJ/m².dia de acordo com dados do CRESESB para Porto Alegre, RS.")

Irradiância total média no plano horizontal é: 15.95 MJ/m².dia de acordo com dados do CRESESB para Porto Alegre, RS.


#Function to calculate the absorbed solar radiation in a period of N_days (Q_s)

In [ ]:
def Absorbed_solar_radiation(A, HSP_MJ, pst, abs, N_days):
    """
    This function calculates the absorbed solar radiation in a period of N_days.
    
    Args:
    - A: area of the swimming pool [m2]
    - HSP_MJ: peak solar hour in MegaJoules (MJ)
    - pst: pool surface transmittance
    - abs: pool absorptance
    - N_days: Number of days
    - L_crv: L is the cube root of the house volume [m]
    
    Returns:
    - Absorbed solar radiation in a period of N_days.
    """
    Q_s = (A * HSP_MJ*pst*abs*N_days)/1000 # absorbed solar radiation (Q_s)
    return Q_s

# Problema proposto 
Uma piscina com as seguintes dimensões (25mx8mx1.5m). Com 200-m2 de área de lâmina d'água, localizada em Porto Alegre, RS. Estime a quantidade de energia requerica para manter a piscina a 28∘C em Abril. A piscina possui cobertura do tipo plastico bolha, que é usada por 12h a noite. Assumir que o vento local é de 3 m/s.

# Input data

In [ ]:
T_a = 16 #float(input("Enter the monthly average ambient temperature [°C]: ")) #monthly average ambient temperature
T_p = 28 #float(input("Enter the desired temperature for the pool [°C]: ")) #monthly average ambient temperature
Ta = T_a + 273.15
Tp = T_p + 273.15
L = 25 #float(input("Enter the length of the pool [m]: "))
W = 8 #float(input("Enter the width of the pool [m]: "))
A = L*W
p = 1.5 # profundidade em metros
pst = 0.92 #pool surface transmittance by Francey and Golding (1981)
abs = 0.6 # pool absorptance by Francey and Golding (1981).

N_days = 30 #número de dias em Abril
N_sec = N_days*24*60*60 #número de segundos em Abril

# Calculating the absorbed solar radiation (Q_s)
Q_s = Absorbed_solar_radiation(A, HSP_MJ, pst, abs, N_days) # absorbed solar radiation (Q_s)
print(f"Absorbed solar radiation is: {Q_s:.2f} GJ")

# Calculating radiation heat losses
K_T = 0.53 # índice de claridade médio mensal
# For dew point temperatures of 10, 20, and 30∘C, the ratio Ts∕Ta is 0.938, 0.961, and 0.986, respectively. The average sky temperature is estimated to be: 
Ts = round(0.938*Ta)
T_mask = round((1-K_T)*Ts+K_T*Ta) # The calculations will begin assuming the pool is uncovered at all times.
sigma = 5.67e-8 # W/m2 K4
N_sec #is the number of seconds in April.
Q_rad = (A*sigma*(Tp**4 - T_mask**4)*N_sec)/(10**9)
print(f"Radiation heat losses is: " +str(round(Q_rad ,2)) + " GJ")

# Calculating evaporation losses
Pa = 101.35 # ambient air pressure in kilopascals (kPa).
V = 2.5 # wind velocity in m/s.
# The monthly average humidity ratio is 0.008 and the saturation humidity ratio for the 25∘C pool is 0.020.

Q_e = (A*Pa*(35*V+43*(Tp-Ta)**(1/3))*(0.020-0.008)*N_sec)/(10**9) # Estimation of evaporation losses of a swimming pool 
print(f"Evaporation losses is: " +str(round(Q_e ,2)) + " GJ")

# Calculating convection heat losses for a swimming pool
Q_c = Q_e*0.0006*((Tp-Ta)/(0.020-0.008)) # The convection losses are found from Equation 9.7.2 of Diffie e Beckmam (2020)
print(f"Convection heat losses is: " +str(round(Q_c ,2)) + " GJ")


# The loads for uncovered pool (Q_Lu), pool with half time film cover (Q_Lfc) an pool with bubble type cover (Q_Lb) are presented hereafter.
Q_Lu = Q_rad + Q_e + Q_c - Q_s # If the pool is uncovered for the whole month, the monthly load is
Q_Lfc = Q_rad + 0.5 *(Q_e) + Q_c - Q_s # If the pool is covered one-half of the time with a film cover, only the evaporation losses will be affected and the monthly load is
Q_Lb = Q_rad + 0.5 *(Q_e + Q_c) - Q_s # If the cover is of the bubble type, the evaporation losses will be eliminated and convection losses will be reasonably small.
print(f"Heat losses for a uncovered pool is: " +str(round(Q_Lu ,2)) + " GJ")
print(f"Heat losses for a film cover pool (one-half of the time): " +str(round(Q_Lfc ,2)) + " GJ")
print(f"Heat losses for a bubble type cover pool: " +str(round(Q_Lb ,2)) + " GJ")

Absorbed solar radiation is: 52.82 GJ
Radiation heat losses is: 58.49 GJ
Evaporation losses is: 117.23 GJ
Convection heat losses is: 70.34 GJ
Heat losses for a uncovered pool is: 193.25 GJ
Heat losses for a film cover pool (one-half of the time): 134.63 GJ
Heat losses for a bubble type cover pool: 99.46 GJ


In [ ]:
# Required inputs
thermal_load = Q_Lu *(10**9/N_sec)  # in watts
collector_efficiency = 0.71 # Eficiência média para coletores para piscina de polipropileno de acordo com tabela PROCEL
collector_size = 2.4  # in square meters

# Calculate the collector area
collector_area = thermal_load / (irrad_total_media * collector_efficiency)

# Determine the number of collectors
num_collectors = round(collector_area / collector_size)

print("Number of solar collectors required: ", num_collectors)

Number of solar collectors required:  103


In [ ]:
import CoolProp.CoolProp as CP
fluid = 'water'
Vol = L*W*p #m3
T_f = (Ta+Tp)/2
rho = CP.PropsSI("D", "T", T_f, "P", Pa * 1e3, fluid)
cp = CP.PropsSI("CPMASS", "T", T_f, "P", Pa * 1e3, fluid)

Q_h = (rho*Vol*cp*(Tp-Ta))/(10**9)
Q_T = Q_h + Q_Lfc
print('The specific mass is: '+str(round(rho ,3))+' kg/m^3')
print('The specific heat is: '+str(round(cp ,3))+' J/kg-K')
print(f"Heat added: " +str(round(Q_h ,2)) + " GJ")

The specific mass is: 997.774 kg/m^3
The specific heat is: 4182.783 J/kg-K
Heat added: 15.02 GJ
